<a href="https://colab.research.google.com/github/hv33y/links-to-onedrive/blob/master/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Magent/Direct (to OneDrive)

In [ ]:
#@title 1. **Install (RClone + Libtorrent)**

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!wget https://downloads.rclone.org/v1.52.1/rclone-v1.52.1-linux-amd64.deb
!apt install ./rclone-v1.52.1-linux-amd64.deb

In [ ]:
#@title 2. **Authenticate Rclone + Onedrive**

!rclone config      #add your onedrive

In [ ]:
#@title 3. **Rclone (Mount OneDrive)**

!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &    

In [ ]:
#@title >>>> __Check Onedrive Mounted__

!ls ./onedrive    #after sucessfully mounted onedrive then show your onedrive folders and files

In [ ]:
#@title 4. **Add Magnet/Torrent**

!mkdir downloads
link = input ("Input Magnet links") 

import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': './downloads/',
    'storage_mode': lt.storage_mode_t(2)}

print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

In [ ]:
#@title **<<<<< yet Experiemental, Download files with direct links.**

!cd downloads && wget -c #paste link here

In [ ]:
#@title 5. **Copy files to OneDrive**
!rsync -ah --progress downloads/ onedrive/            #this may take a while 